# 📊 数据预处理模块 (0_prep.ipynb)

## 🎯 模块目标
本模块负责原始数据的加载、清洗和预处理，为后续的召回和排序模块提供标准化的数据输入。

## 📋 主要功能
1. **数据加载**: 读取训练集、测试集和商品属性数据
2. **数据排序**: 按用户ID、时间、排序字段进行标准化排序
3. **数据验证**: 检查必要字段的完整性和正确性
4. **留一验证**: 科学的时序切分，避免数据泄漏
5. **数据保存**: 生成标准化的中间文件供后续使用

## 🔧 输出文件
- `train_sorted.parquet`: 排序后的完整训练数据
- `test_sorted.parquet`: 排序后的测试数据  
- `train_vis.parquet`: 用于离线统计的训练数据（去除标签）
- `label_df.parquet`: 留一验证的标签数据
- `item_attr.parquet`: 清洗后的商品属性数据


## 1️⃣ 环境配置与依赖导入


In [ ]:
# =============================================================================
# 依赖库导入
# =============================================================================
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime

# =============================================================================
# 配置参数
# =============================================================================
# 数据文件路径
TRAIN_CSV = '../data/Antai_hackathon_train.csv'
TEST_CSV  = '../data/dianshang_test.csv'
ATTR_CSV  = '../data/Antai_hackathon_attr.csv'

# 输出目录
OUTDIR = '../x'
os.makedirs(OUTDIR, exist_ok=True)

print("✅ 环境配置完成")
print(f"📁 输出目录: {OUTDIR}")
print(f"⏰ 处理时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ 环境配置完成
📁 输出目录: ../x
⏰ 处理时间: 2025-09-10 21:36:24


## 2️⃣ 数据类型定义与加载


In [ ]:
# =============================================================================
# 数据类型定义 - 内存优化
# =============================================================================
dtype_train = {
    "buyer_admin_id": "int32",    # 用户ID
    "item_id": "int32",           # 商品ID
    "irank": "int16"              # 排序字段
}

dtype_test = {
    "buyer_admin_id": "int32",
    "item_id": "int32",
    "irank": "int16"
}

dtype_item_attr = {
    "item_id": "int32",
    "cate_id": "int32", 
    "store_id": "int32"
}

print("📋 数据类型定义完成")
print("💾 使用int32/int16减少内存占用")


📋 数据类型定义完成
💾 使用int32/int16减少内存占用


In [ ]:
# =============================================================================
# 数据加载
# =============================================================================
print("🔄 正在加载数据文件...")

# 加载训练数据
print("  📖 加载训练数据...")
train = pd.read_csv(
    TRAIN_CSV,
    parse_dates=['create_order_time'],  # 时间字段自动解析
    dtype=dtype_train
)

# 加载测试数据  
print("  📖 加载测试数据...")
test = pd.read_csv(
    TEST_CSV,
    parse_dates=['create_order_time'],
    dtype=dtype_test
)

# 加载商品属性数据
print("  📖 加载商品属性数据...")
item_attr = pd.read_csv(
    ATTR_CSV,
    dtype=dtype_item_attr
)

print("✅ 数据加载完成")
print(f"📊 训练集: {train.shape}")
print(f"📊 测试集: {test.shape}")  
print(f"📊 商品属性: {item_attr.shape}")


🔄 正在加载数据文件...
  📖 加载训练数据...
  📖 加载测试数据...
  📖 加载商品属性数据...
✅ 数据加载完成
📊 训练集: (6989817, 5)
📊 测试集: (140380, 5)
📊 商品属性: (1924269, 4)


## 3️⃣ 数据质量检查


In [ ]:
# =============================================================================
# 数据完整性检查
# =============================================================================
print("🔍 开始数据质量检查...")

# 定义必需字段
REQUIRED_TRAIN_COLS = {'buyer_admin_id', 'item_id', 'create_order_time', 'irank'}
REQUIRED_ATTR_COLS = {'item_id', 'cate_id', 'store_id'}

# 检查训练数据字段
missing_train = REQUIRED_TRAIN_COLS - set(train.columns)
missing_test = REQUIRED_TRAIN_COLS - set(test.columns)
missing_attr = REQUIRED_ATTR_COLS - set(item_attr.columns)

# 断言检查
assert len(missing_train) == 0, f'❌ 训练数据缺少字段: {missing_train}'
assert len(missing_test) == 0, f'❌ 测试数据缺少字段: {missing_test}'
assert len(missing_attr) == 0, f'❌ 商品属性缺少字段: {missing_attr}'

print("✅ 数据字段完整性检查通过")

# 基础统计信息
print("\n📈 数据基础统计:")
print(f"  👥 用户数量: {train['buyer_admin_id'].nunique():,}")
print(f"  🛍️ 商品数量: {train['item_id'].nunique():,}")
print(f"  📦 商品属性数量: {item_attr.shape[0]:,}")
print(f"  📅 时间范围: {train['create_order_time'].min()} ~ {train['create_order_time'].max()}")

# 检查缺失值
print(f"\n🔍 缺失值检查:")
print(f"  训练数据缺失值: {train.isnull().sum().sum()}")
print(f"  测试数据缺失值: {test.isnull().sum().sum()}")
print(f"  商品属性缺失值: {item_attr.isnull().sum().sum()}")


🔍 开始数据质量检查...
✅ 数据字段完整性检查通过

📈 数据基础统计:
  👥 用户数量: 483,117
  🛍️ 商品数量: 1,852,506
  📦 商品属性数量: 1,924,269
  📅 时间范围: 2018-03-13 04:01:00 ~ 2018-04-28 23:59:57

🔍 缺失值检查:
  训练数据缺失值: 0
  测试数据缺失值: 0
  商品属性缺失值: 0


## 4️⃣ 数据排序与标准化


In [ ]:
# =============================================================================
# 数据排序标准化
# =============================================================================
print("🔄 正在进行数据排序...")

# 确保数据按时序正确排序
# 排序规则: 用户ID -> 时间 -> irank (升序)
train = train.sort_values(
    ['buyer_admin_id', 'create_order_time', 'irank']
).reset_index(drop=True)

test = test.sort_values(
    ['buyer_admin_id', 'create_order_time', 'irank']
).reset_index(drop=True)

print("✅ 数据排序完成")
print("📋 排序字段: buyer_admin_id → create_order_time → irank")


🔄 正在进行数据排序...
✅ 数据排序完成
📋 排序字段: buyer_admin_id → create_order_time → irank


## 5️⃣ 留一验证切分


In [ ]:
# =============================================================================
# 留一验证切分 - 避免数据泄漏
# =============================================================================
print("🔄 开始留一验证切分...")

# 找到每个用户的最后一条购买记录作为标签
# 注意: 这里使用idxmin()是因为排序后第一条是最早的记录
# 我们需要最后一条记录，所以使用idxmax()
last_idx = train.groupby('buyer_admin_id')['irank'].idxmax()

# 创建标签数据集
label_df = train.loc[last_idx, ['buyer_admin_id', 'item_id']].copy()
label_df = label_df.rename(columns={'item_id': 'label_item'})

# 创建训练可视数据集（去除标签数据，防止泄漏）
train_vis = train.drop(index=last_idx).copy()

print("✅ 留一验证切分完成")
print(f"📊 原始训练数据: {train.shape[0]:,} 条")
print(f"📊 训练可视数据: {train_vis.shape[0]:,} 条")
print(f"📊 标签数据: {label_df.shape[0]:,} 条")
print(f"👥 参与验证的用户: {label_df['buyer_admin_id'].nunique():,} 人")


🔄 开始留一验证切分...
✅ 留一验证切分完成
📊 原始训练数据: 6,989,817 条
📊 训练可视数据: 6,506,700 条
📊 标签数据: 483,117 条
👥 参与验证的用户: 483,117 人


## 6️⃣ 数据保存与输出


In [ ]:
# =============================================================================
# 数据保存 - 生成标准化的中间文件
# =============================================================================
print("💾 正在保存处理后的数据...")

# 保存训练数据（完整版）
train.to_parquet(f'{OUTDIR}/train_sorted.parquet', index=False)
print(f"  ✅ 保存完整训练数据: train_sorted.parquet")

# 保存测试数据
test.to_parquet(f'{OUTDIR}/test_sorted.parquet', index=False)
print(f"  ✅ 保存测试数据: test_sorted.parquet")

# 保存商品属性数据（去重后）
item_attr_clean = item_attr[['item_id', 'cate_id', 'store_id']].drop_duplicates()
item_attr_clean.to_parquet(f'{OUTDIR}/item_attr.parquet', index=False)
print(f"  ✅ 保存商品属性数据: item_attr.parquet")

# 保存训练可视数据（用于离线统计）
train_vis.to_parquet(f'{OUTDIR}/train_vis.parquet', index=False)
print(f"  ✅ 保存训练可视数据: train_vis.parquet")

# 保存标签数据
label_df.to_parquet(f'{OUTDIR}/label_df.parquet', index=False)
print(f"  ✅ 保存标签数据: label_df.parquet")

print(f"\n🎉 数据预处理完成！")
print(f"📁 所有文件已保存到: {OUTDIR}")
print(f"📊 最终数据规模总结:")
print(f"  - 完整训练数据: {train.shape}")
print(f"  - 测试数据: {test.shape}")
print(f"  - 训练可视数据: {train_vis.shape}")
print(f"  - 标签数据: {label_df.shape}")
print(f"  - 商品属性数据: {item_attr_clean.shape}")


💾 正在保存处理后的数据...
  ✅ 保存完整训练数据: train_sorted.parquet
  ✅ 保存测试数据: test_sorted.parquet
  ✅ 保存商品属性数据: item_attr.parquet
  ✅ 保存训练可视数据: train_vis.parquet
  ✅ 保存标签数据: label_df.parquet

🎉 数据预处理完成！
📁 所有文件已保存到: ../x
📊 最终数据规模总结:
  - 完整训练数据: (6989817, 5)
  - 测试数据: (140380, 5)
  - 训练可视数据: (6506700, 5)
  - 标签数据: (483117, 2)
  - 商品属性数据: (1924269, 3)
